In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.pesmaster.com/russian-premier-liga/pes-2020/league/114/'
response = requests.get(url)

In [3]:
soup = BeautifulSoup(response.text,'html.parser')

In [4]:
team_links = []
for data in soup.findAll('div', {'class': 'team-block'}):
    for a in data.find_all('a'):
        team_links.append(a.get('href'))

In [5]:
team_links

['/akhmat-grozny/pes-2020/team/5196/',
 '/arsenal-tula/pes-2020/team/5197/',
 '/cska-moskva/pes-2020/team/1217/',
 '/dinamo-moskva/pes-2020/team/1753/',
 '/krasnodar/pes-2020/team/2618/',
 '/krylya-sovetov/pes-2020/team/4143/',
 '/lokomotiv-moskva/pes-2020/team/271/',
 '/orenburg/pes-2020/team/5301/',
 '/rostov/pes-2020/team/2229/',
 '/rubin-kazan/pes-2020/team/1941/',
 '/sochi/pes-2020/team/5296/',
 '/spartak-moskva/pes-2020/team/135/',
 '/tambov/pes-2020/team/5306/',
 '/ufa/pes-2020/team/5200/',
 '/ural/pes-2020/team/5201/',
 '/zenit/pes-2020/team/1218/']

In [6]:
teams = [i.split('/')[1] for i in team_links]

In [7]:
teams

['akhmat-grozny',
 'arsenal-tula',
 'cska-moskva',
 'dinamo-moskva',
 'krasnodar',
 'krylya-sovetov',
 'lokomotiv-moskva',
 'orenburg',
 'rostov',
 'rubin-kazan',
 'sochi',
 'spartak-moskva',
 'tambov',
 'ufa',
 'ural',
 'zenit']

In [8]:
start_addres = 'https://www.pesmaster.com'
team_links = [start_addres + i for i in team_links]

In [9]:
team_links

['https://www.pesmaster.com/akhmat-grozny/pes-2020/team/5196/',
 'https://www.pesmaster.com/arsenal-tula/pes-2020/team/5197/',
 'https://www.pesmaster.com/cska-moskva/pes-2020/team/1217/',
 'https://www.pesmaster.com/dinamo-moskva/pes-2020/team/1753/',
 'https://www.pesmaster.com/krasnodar/pes-2020/team/2618/',
 'https://www.pesmaster.com/krylya-sovetov/pes-2020/team/4143/',
 'https://www.pesmaster.com/lokomotiv-moskva/pes-2020/team/271/',
 'https://www.pesmaster.com/orenburg/pes-2020/team/5301/',
 'https://www.pesmaster.com/rostov/pes-2020/team/2229/',
 'https://www.pesmaster.com/rubin-kazan/pes-2020/team/1941/',
 'https://www.pesmaster.com/sochi/pes-2020/team/5296/',
 'https://www.pesmaster.com/spartak-moskva/pes-2020/team/135/',
 'https://www.pesmaster.com/tambov/pes-2020/team/5306/',
 'https://www.pesmaster.com/ufa/pes-2020/team/5200/',
 'https://www.pesmaster.com/ural/pes-2020/team/5201/',
 'https://www.pesmaster.com/zenit/pes-2020/team/1218/']

In [170]:
from collections import defaultdict
pl_dict = defaultdict(dict)

In [171]:
# собрали словарь ссылок на всех игроков
for n, link in enumerate(team_links):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    for data in soup.find_all(id='search-result-table'):
        player_links = []
        for a in data.find_all('a', class_='namelink'):
            player_links.append(a.get('href'))
        players = [i.split('/')[1] for i in player_links]
        player_links = [start_addres + i for i in player_links]
        for k, pl in enumerate(players):
            pl_dict[teams[n]][pl] = player_links[k]

In [172]:
def amplua_encoder(amp):
    if amp == 'GK':
        return 1
    if amp in ('CB', 'RB', 'LB'):
        return 2
    if amp == 'DMF':
        return 3
    if amp in ('CMF', 'LMF', 'RMF'):
        return 4
    if amp in ('AMF', 'SS'):
        return 5
    if amp in ('CF', 'LWF', 'RWF'):
        return 6
    if amp == 'Right':
        return 1
    if amp == 'Left':
        return -1
    if not amp.isdigit():
        return 0
    return amp

In [173]:
def load_player(link):
    response = requests.get(link)
    soap = BeautifulSoup(response.text, 'lxml')
    features_dict = {}
    for data in soap.find_all('div', {'class' : 'stats-block'}):
        if data.span.text is not None:
            features_dict[data.h4.contents[1]] = float(data.span.text)
        for row in data.find('tbody').find_all('tr'):
            values = row.find_all('td')
            features_dict[values[1].text] = float(values[0].text)
        for pos_row in soap.find_all('div', {'class' : 'player-positions-row'}):
            for item in pos_row.find_all('div'):
                values = item.find_all('span')
                features_dict[values[0].text] = float(values[1].text)
        for pl_info_row in soap.find('table', {'class' : 'player-info'}).find_all('tr'):
            values = pl_info_row.find_all('td')
            if values[0].text not in ('Position', 'Age', 'Stronger Foot', 'Height', 'Weight'):
                continue
            features_dict[values[0].text] = float(amplua_encoder(values[1].text))
    return features_dict

In [174]:
for team in pl_dict:
    for pl, link in pl_dict[team].items():
        pl_dict[team][pl] = load_player(link)

In [177]:
len(pl_dict['zenit']['m-kerzhakov'])

52

In [176]:
with open('russian-premier-league.csv', 'w+') as f:
    f.write('Team, Player, ' + ', '.join(pl_dict['zenit']['a-dzyuba'].keys()))
    f.write('\n')
    for key in pl_dict:
        for player in pl_dict[key]:
            f.write(key + ', ' + player + ', ' + ', '.join([str(x) for x in pl_dict[key][player].values()]))
            f.write('\n')

In [178]:
лиза = 10
игорь = 15

In [179]:
лиза * игорь

150

In [180]:
игорь / лиза

1.5

In [181]:
игорь ** лиза

576650390625